In [5]:
import pandas as pd
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.model_selection import cross_val_score

In [2]:
df=pd.read_csv('/Users/richard/Desktop/Final_Data.csv')

In [6]:
y=df['Score']
DF1=df.drop(['Name','Nationality','Score','friends','solo', 'Date_new'],axis=1)

In [16]:
df_new = df[DF1['Topic'].notnull()]
df_new.to_csv('/Users/richard/Desktop/Final_Data2.csv')

In [7]:
def remove_punctuation(text):
    '''a function for removing punctuation'''
    import string
    # replacing the punctuations with no space, 
    # which in effect deletes the punctuation marks 
    translator = str.maketrans('', '', string.punctuation)
    # return the text stripped of punctuation marks
    return text.translate(translator)

sw = stopwords.words('english')
def stopwords(text):
    '''a function for removing the stopword'''
    # removing the stop words and lowercasing the selected words
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    # joining the list of words with space separator
    return " ".join(text)

stemmer = SnowballStemmer("english")
def stemming(text):    
    '''a function which stems each word in the given text'''
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text)

In [18]:
df_new['Topic']=df_new['Topic'].apply(remove_punctuation)
df_new['Topic']=df_new['Topic'].apply(stopwords)
df_new['Topic']=df_new['Topic'].apply(stemming)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [20]:
cv = CountVectorizer()
data = cv.fit_transform(df_new.Topic)

In [21]:
DF2=pd.concat([pd.DataFrame(data.toarray()), df_new], axis=1)

In [23]:
DF2.drop(['Name','Nationality','Score','friends','solo', 'Date_new','Topic'],axis=1,inplace=True)

In [25]:
y_new=[]
for i in y:
    if i>=8:
        a=2
    elif 6<i<8:
        a=1
    else:
        a=0
    y_new.append(a)

In [30]:
DF2=DF2.fillna('0')

In [31]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(DF2,y_new,test_size=0.1,random_state=42)

In [32]:
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import accuracy_score
lr = LogisticRegression()
param_grid = {'C': np.linspace(1, 100, 1), 'penalty': ['l1', 'l2']}
grid_lr = GridSearchCV(lr, param_grid=param_grid, cv=10, n_jobs=1)

In [33]:
grid_lr.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': array([ 1.]), 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [34]:
lr=grid_lr.best_estimator_
lr.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [35]:
y_pred_lr=lr.predict(x_test)
from sklearn.metrics import confusion_matrix
LRcm=confusion_matrix(y_test,y_pred_lr)
print(LRcm)

[[ 15   1  26]
 [ 10   6  91]
 [  5   6 401]]


In [36]:
precision=LRcm.trace()/LRcm.sum()
print(precision)

0.752228163993


In [37]:
from sklearn.ensemble import RandomForestClassifier
#sample_leaf_options = [1,5,10,50,100,200,500]
#for leaf_size in sample_leaf_options:
    #model = RandomForestClassifier(n_estimators = 200, oob_score = True, n_jobs = -1,random_state =50,max_features = "auto", min_samples_leaf = leaf_size)
    #model.fit(DF2,y_new)
    #print(cross_val_score(model, X=DF2, y=y_new, cv=10, scoring='accuracy').mean())
classifier=RandomForestClassifier(random_state=0)

In [40]:
#tuning parameters
from sklearn.model_selection import GridSearchCV
n_estimators_range=[10,100,500,1000]
max_depth_range=[None,5,8,15,30]
min_samples_split_range=[2,5,10,50,100]
min_samples_leaf_range=[1,2,5,10]
param_grid = dict(n_estimators=n_estimators_range, max_depth=max_depth_range, min_samples_leaf=min_samples_leaf_range, min_samples_split=min_samples_split_range)
grid = GridSearchCV(estimator=classifier, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(x_train,y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
#for mean, stdev, param in zip(means, stds, params):
    #print("%f (%f) with: %r" % (mean, stdev, param))
#(Result is too long on github)

Best: 0.748811 using {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 500}


In [41]:
rfc_model=grid.best_estimator_
rfc_model.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [42]:
y_pred_rfc=rfc_model.predict(x_test)
cm_rfc=confusion_matrix(y_test,y_pred_rfc)
print(cm_rfc)

[[ 12   2  28]
 [  6  11  90]
 [  4   8 400]]


In [43]:
precision_rfc=cm_rfc.trace()/cm_rfc.sum()
print(precision_rfc)

0.754010695187


In [48]:
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import xgboost as xgb
from skopt import BayesSearchCV
from sklearn.model_selection import StratifiedKFold

# SETTINGS - CHANGE THESE TO GET SOMETHING MEANINGFUL
ITERATIONS = 10 # 1000
TRAINING_SIZE = 100000 # 20000000
TEST_SIZE = 25000

In [45]:
x_train['Review_times']=pd.to_numeric(x_train['Review_times'])
x_test['Review_times']=pd.to_numeric(x_test['Review_times'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [49]:
bayes_cv_tuner = BayesSearchCV(
    estimator = xgb.XGBClassifier(
        n_jobs = 1,
        objective = 'multi:softmax',
        silent=1,
        tree_method='approx'
    ),
    search_spaces = {
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'min_child_weight': (0, 10),
        'max_depth': (0, 50),
        'max_delta_step': (0, 20),
        'subsample': (0.01, 1.0, 'uniform'),
        'colsample_bytree': (0.01, 1.0, 'uniform'),
        'colsample_bylevel': (0.01, 1.0, 'uniform'),
        'reg_lambda': (1e-9, 1000, 'log-uniform'),
        'reg_alpha': (1e-9, 1.0, 'log-uniform'),
        'gamma': (1e-9, 0.5, 'log-uniform'),
        'min_child_weight': (0, 5),
        'n_estimators': (50, 100),
        'scale_pos_weight': (1e-6, 500, 'log-uniform')
    },    
    cv = StratifiedKFold(
        n_splits=5,
        shuffle=True,
        random_state=42
    ),
    n_jobs = 3,
    n_iter = ITERATIONS,   
    verbose = 0,
    refit = True,
    random_state = 42
)

def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))
    
    # Save all model results
    clf_name = bayes_cv_tuner.estimator.__class__.__name__
    all_models.to_csv(clf_name+"_cv_results.csv")

In [53]:
result = bayes_cv_tuner.fit(x_train.values, y_train, callback=status_print)

Model #1
Best ROC-AUC: 0.7429
Best params: {'colsample_bylevel': 0.41600291926478072, 'colsample_bytree': 0.73044848574555188, 'gamma': 0.13031389926541354, 'learning_rate': 0.042815319280763466, 'max_delta_step': 13, 'max_depth': 21, 'min_child_weight': 2, 'n_estimators': 87, 'reg_alpha': 5.4975577392897861e-07, 'reg_lambda': 0.059360706359120489, 'scale_pos_weight': 0.060830282487222144, 'subsample': 0.13556548021189216}

Model #2
Best ROC-AUC: 0.746
Best params: {'colsample_bylevel': 0.83901447199775159, 'colsample_bytree': 0.88448212460705367, 'gamma': 4.3586846084807948e-07, 'learning_rate': 0.79881794627812419, 'max_delta_step': 17, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 68, 'reg_alpha': 0.00052669830037015467, 'reg_lambda': 276.54244755742252, 'scale_pos_weight': 0.30164107718431421, 'subsample': 0.99237105986371343}

Model #3
Best ROC-AUC: 0.746
Best params: {'colsample_bylevel': 0.83901447199775159, 'colsample_bytree': 0.88448212460705367, 'gamma': 4.3586846084

In [59]:
xgb_model=XGBClassifier(colsample_bylevel=0.83901447199775159, colsample_bytree=0.88448212460705367, 
                        gamma= 4.3586846084807948e-07, learning_rate= 0.79881794627812419, 
                        max_delta_step=17, max_depth= 3, min_child_weight=1, 
                        n_estimators= 68, reg_alpha= 0.00052669830037015467, reg_lambda= 276.54244755742252, 
                        scale_pos_weight=0.30164107718431421, subsample=0.99237105986371343)

In [60]:
xgb_model.fit(x_train.values, y_train)

XGBClassifier(base_score=0.5, booster='gbtree',
       colsample_bylevel=0.8390144719977516,
       colsample_bytree=0.8844821246070537, gamma=4.358684608480795e-07,
       learning_rate=0.7988179462781242, max_delta_step=17, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=68, n_jobs=1,
       nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0.0005266983003701547, reg_lambda=276.5424475574225,
       scale_pos_weight=0.3016410771843142, seed=None, silent=True,
       subsample=0.9923710598637134)

In [61]:
y_pred_xgboost=xgb_model.predict(x_test.values)
from sklearn.metrics import confusion_matrix
XGboost_cm=confusion_matrix(y_test,y_pred_xgboost)
print(XGboost_cm)

[[ 11   6  25]
 [  8  17  82]
 [  5  21 386]]


In [62]:
precision_xgboost=XGboost_cm.trace()/XGboost_cm.sum()
print(precision_xgboost)

0.737967914439
